In [1]:
from collections import deque

In [2]:
class Client:
    def __init__(self, name: str, priority: str, arrival_time: int):
        """
        name: имя (или иной идентификатор) клиента
        priority: приоритет, например, 'VIP' или 'normal'
        arrival_time: условное время прихода (в минутах с начала работы, либо счётчик)
        """
        self.name = name
        self.priority = priority
        self.arrival_time = arrival_time
        self.start_service_time = None
        self.end_service_time = None

    @property
    def wait_time(self):
        """Сколько времени клиент ждал до начала обслуживания"""
        if self.start_service_time is not None:
            return self.start_service_time - self.arrival_time
        return None

    @property
    def service_time(self):
        """Сколько времени заняло обслуживание у клиента"""
        if self.start_service_time is not None and self.end_service_time is not None:
            return self.end_service_time - self.start_service_time
        return None

    def __repr__(self):
        return f"<Client {self.name}, prio={self.priority}>"


class ElectronicQueue:
    def __init__(self):
        # Отдельные очереди для VIP и обычных
        self.vip_queue = deque()
        self.normal_queue = deque()

        # Список обслуженных клиентов — чтобы сформировать отчет
        self.served_clients = []

        # Условное "время" внутри системы (например, минуты с момента открытия)
        self.current_time = 0

    def add_client(self, name: str, priority: str):
        """
        Добавляет клиента в очередь.
        Параллельно фиксируем время прихода = self.current_time
        и искусственно "двигаем" внутренний таймер (на 1 единицу).
        """
        client = Client(name, priority, arrival_time=self.current_time)
        if priority.lower() == "vip":
            self.vip_queue.append(client)
        else:
            self.normal_queue.append(client)

        # Считаем, что между поступлениями клиентов проходит 1 условная минута
        self.current_time += 1

    def serve_next(self, service_time: int = 5):
        """
        Обслуживает следующего клиента.
        service_time — условное время обслуживания (в минутах).
        Если VIP-очередь не пуста, берём оттуда, иначе — из обычной.
        """
        if not self.vip_queue and not self.normal_queue:
            print("Нет клиентов в очереди.")
            return

        # Сначала обслуживаем VIP, если есть
        if self.vip_queue:
            client = self.vip_queue.popleft()
        else:
            client = self.normal_queue.popleft()

        # Время начала обслуживания — текущий момент
        client.start_service_time = self.current_time
        # Предположим, всё обслуживание занимает service_time минут
        client.end_service_time = self.current_time + service_time

        # Сдвигаем "часы" на время обслуживания
        self.current_time += service_time

        # Сохраняем в список обслуженных
        self.served_clients.append(client)
        print(f"Обслужили {client.name} (приоритет={client.priority}).")

    def serve_all(self, service_time_per_client: int = 5):
        """
        Упрощённый метод: обслужить всех клиентов,
        пока очереди не опустеют.
        """
        while self.vip_queue or self.normal_queue:
            self.serve_next(service_time=service_time_per_client)

    def get_report(self):
        """
        Формируем отчёт о времени обслуживания клиентов:
        - Сколько всего человек обслужено
        - Среднее время ожидания
        - Среднее время обслуживания (заданное сервисом)
        - Общее время обслуживания (от первого до последнего)
        """
        if not self.served_clients:
            return "Никто не был обслужен."

        total_clients = len(self.served_clients)
        total_wait_time = 0
        total_service_time = 0

        for c in self.served_clients:
            total_wait_time += (c.wait_time or 0)
            total_service_time += (c.service_time or 0)

        avg_wait = total_wait_time / total_clients
        avg_service = total_service_time / total_clients

        # Время от arrival_time первого клиента до завершения последнего
        # (Можно уточнить логику: у нас current_time в конце = время окончания)
        overall_duration = self.served_clients[-1].end_service_time - self.served_clients[0].arrival_time

        report = (
            f"Отчёт по обслуживанию:\n"
            f" - Всего обслужено клиентов: {total_clients}\n"
            f" - Среднее время ожидания (минуты): {avg_wait:.2f}\n"
            f" - Среднее время обслуживания (минуты): {avg_service:.2f}\n"
            f" - Общее время (от прихода первого до конца последнего): {overall_duration} минут\n"
        )
        return report

In [3]:
# Пример использования
if __name__ == "__main__":
    queue = ElectronicQueue()

    # Добавляем несколько клиентов
    queue.add_client("Иван", priority="VIP")
    queue.add_client("Петр", priority="normal")
    queue.add_client("Ольга", priority="normal")
    queue.add_client("Мария", priority="VIP")
    queue.add_client("Алексей", priority="normal")

    # Обслуживаем всех
    queue.serve_all(service_time_per_client=5)

    # Выводим отчёт
    print(queue.get_report())

Обслужили Иван (приоритет=VIP).
Обслужили Мария (приоритет=VIP).
Обслужили Петр (приоритет=normal).
Обслужили Ольга (приоритет=normal).
Обслужили Алексей (приоритет=normal).
Отчёт по обслуживанию:
 - Всего обслужено клиентов: 5
 - Среднее время ожидания (минуты): 13.00
 - Среднее время обслуживания (минуты): 5.00
 - Общее время (от прихода первого до конца последнего): 30 минут

